In [ ]:
import os
import ase.io
import re
import pandas as pd
import numpy as np
from dscribe.descriptors import SOAP
from dscribe.descriptors import MBTR,ACSF,CoulombMatrix,EwaldSumMatrix,SineMatrix 
import tensorflow as tf
from tensorflow.keras.models import Model
from numpy                 import array
from sklearn               import metrics
from sklearn.preprocessing import MinMaxScaler
from keras.models          import Sequential
from keras.layers          import *
from sklearn               import metrics
from tensorflow.keras import layers
import keras
from keras.layers import Conv1D, MaxPooling1D, Dense, Flatten,Conv2D, MaxPool2D,LSTM,Bidirectional
from keras.models import Sequential
from keras import Input
from keras.layers          import *
from keras.models import save_model,load_model
import time
#####decribe parameters#####
rcut = 6
#####decribe parameters#####
sm = SineMatrix(
    n_atoms_max=120,
    permutation="sorted_l2",
    sparse=False,
    flatten=False)
ew=EwaldSumMatrix(
    n_atoms_max=120,
    permutation="none",flatten=False,sparse=False)
acsf=ACSF(species=['Eu','Sc','V','Cu','Ir','Pt','Au','O'],
    rcut=rcut,
    periodic=True,
    g2_params=[[1, 1], [1, 2], [1, 3]],
    g4_params=[[1, 1, 1], [1, 2, 1], [1, 1, -1], [1, 2, -1]])

In [ ]:
def find_energy(content):
    number=re.findall('\d+',content[-1])
    splitted=content[-1].split()
    energy=float(splitted[4])
    return energy
def read_energy(dir_list,describer1,describer2,describer3):
    list_input=[]
    list_energy_SLAB=[]
    i=0
    for dir_each in dir_list:
        try:
            time_start=time.time()
            dir_each=path+dir_each
            Read_Struct_SLAB=ase.io.read(dir_each+'/POSCAR_BACKUP_INIT')
            des1=describer1.create(Read_Struct_SLAB,n_jobs=-1)
            des2=describer2.create(Read_Struct_SLAB,n_jobs=-1)
            des3=describer3.create(Read_Struct_SLAB,n_jobs=-1)
            new_des=np.concatenate((des1,des2),axis=1)
            new_des=np.concatenate((new_des,des3),axis=1)
            print(new_des.shape)
            new_des=new_des.reshape(240,208)
            with open(dir_each+'/OSZICAR',"r") as SLAB:
                content_SLAB=SLAB.read().splitlines()
            #####################################
            energy_SLAB=find_energy(content_SLAB)
            if energy_SLAB<-700:
                list_input.append(new_des)
                list_energy_SLAB.append(energy_SLAB)
            else:
                print(dir_each)
                pass
            i+=1
            time_finish=time.time()
            print(i)
            print('time cost',time_finish-time_start,'s')
        except Exception as e:
            print('folder not exist')
            print(e)
    return list_input,list_energy_SLAB

In [ ]:
#####computed on cluster#####
path="./result_DFT_HEO_Neutral/"
############################
dir_list=os.listdir(path)
list_input,list_energy_SLAB=read_energy(dir_list,sm,ew,acsf)

In [ ]:
######specify the input and output######
data_input=np.around(list_input,decimals=8)
data_output_SLAB=np.around(list_energy_SLAB,decimals=8)

In [ ]:
from numpy import save
np.save('data_input_sm_ew_acsf_rcu6.npy',data_input)
np.save('data_output_sm_ew_acsf_rcu6_SLAB.npy',data_output_SLAB)